In [67]:
from teradataml import *
import sys
import getpass
import logging


logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [68]:
import matplotlib.pyplot as plt
from collections import Counter

In [6]:
host = 'vantage24.td.teradata.com'
username = 'AJ255081'
password = getpass.getpass("Password: ")

eng = create_context(host=host, username=username, password=password, logmech="TDNEGO")
conn = eng.connect()

# configure byom/val installation
configure.val_install_location = 'val'
configure.byom_install_location = 'mldb'

In [7]:
db_list_tables()

c:\Users\aj255081\AppData\Local\miniconda3\envs\mlops_kt\lib\site-packages\teradataml\dbutils\dbutils.py:333: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pddf = pd.read_sql(query, tdmlctx.td_connection.connection)
c:\Users\aj255081\AppData\Local\miniconda3\envs\mlops_kt\lib\site-packages\teradataml\dbutils\dbutils.py:339: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vtdf = pd.read_sql(vtquery, tdmlctx.td_connection.connection)


,TableName
0,aoa_byom_models
1,aoa_statistics_metadata
2,pima_patient_predictions
3,pima_patient_diagnoses
4,pima_patient_features


In [8]:
DataFrame("pima_patient_diagnoses")

PatientId,HasDiabetes
343,0
28,0
476,1
367,0
712,1
39,1
447,0
648,1
282,0
671,0


In [9]:
train_df = DataFrame.from_query("""
    SEL f.*, d.HasDiabetes 
    FROM aj255081.pima_patient_features f
    INNER JOIN aj255081.pima_patient_diagnoses d
        ON f.PatientId = d.PatientId;
""")
train_df

PatientId,NumTimesPrg,PlGlcConc,BloodP,SkinThick,TwoHourSerIns,BMI,DiPedFunc,Age,HasDiabetes
467,0,97,64,36,100,36.8,0.6,25,0
406,4,115,72,0,0,28.9,0.376,46,1
507,1,130,60,23,170,28.6,0.6920000000000001,21,0
753,0,181,88,44,510,43.3,0.222,26,1
122,2,107,74,30,100,33.6,0.4039999999999999,23,0
671,1,99,58,10,0,25.4,0.551,21,0
326,1,122,64,32,156,35.1,0.6920000000000001,30,1
703,2,129,0,0,0,38.5,0.304,41,0
141,5,106,82,30,0,39.5,0.286,38,0
530,2,122,60,18,106,29.8,0.7170000000000001,22,0


In [10]:
feature_names = ['NumTimesPrg', 'PlGlcConc', 'BloodP', 'SkinThick', 'TwoHourSerIns', 'BMI', 'DiPedFunc', 'Age']
target_name = 'HasDiabetes'
entity_key = 'PatientId'

scale_method = 'RANGE'
miss_value = 'KEEP'
global_scale = False
multiplier = "1"
intercept = "0"

scaler = ScaleFit(
    data=train_df,
    target_columns=feature_names,
    scale_method=scale_method,
    miss_value=miss_value,
    global_scale=global_scale,
    multiplier=multiplier,
    intercept=intercept
)

In [11]:
scaled_train = ScaleTransform(
    data=train_df,
    object=scaler.output,
    accumulate=[target_name, entity_key]
)

In [23]:
tree_type = 'CLASSIFICATION'
max_depth = 12
num_trees = 4

model = DecisionForest(
    data=scaled_train.result,
    input_columns=feature_names,
    response_column=target_name,
    tree_type=tree_type,
    max_depth=max_depth  
)

In [27]:
model_pdf = model.result.to_pandas()['classification_tree']

In [25]:
def traverse_tree(tree, feature_counter):
    if 'split_' in tree and 'attr_' in tree['split_']:
        feature_counter[tree['split_']['attr_']] += 1
    if 'leftChild_' in tree:
        traverse_tree(tree['leftChild_'], feature_counter)
    if 'rightChild_' in tree:
        traverse_tree(tree['rightChild_'], feature_counter)


def compute_feature_importance(trees_json):
    feature_counter = Counter()
    for tree_json in trees_json:
        tree = json.loads(tree_json)
        traverse_tree(tree, feature_counter)
    total_splits = sum(feature_counter.values())
    feature_importance = {
        feature: count / total_splits for feature, count in feature_counter.items()}
    return feature_importance


def plot_feature_importance(fi, img_filename):
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(
        kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

In [28]:
feature_importance = compute_feature_importance(model_pdf)
feature_importance

{'Age': 0.10638297872340426,
 'SkinThick': 0.09574468085106383,
 'BMI': 0.09574468085106383,
 'PlGlcConc': 0.35106382978723405,
 'BloodP': 0.10638297872340426,
 'NumTimesPrg': 0.14893617021276595,
 'TwoHourSerIns': 0.05319148936170213,
 'DiPedFunc': 0.0425531914893617}

In [ ]:
plot_feature_importance(
        feature_importance, "feat_imp")

<Figure size 640x480 with 0 Axes>

In [ ]:
model


############ result Output ############

   task_index  tree_num  tree_order                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [19]:
model_pdf

0     {"id_":1,"size_":22,"maxDepth_":12,"responseCo...
1     {"id_":1,"size_":26,"maxDepth_":12,"responseCo...
2     {"id_":1,"size_":17,"maxDepth_":12,"responseCo...
3     {"id_":1,"size_":18,"maxDepth_":12,"responseCo...
4     {"id_":1,"size_":35,"maxDepth_":12,"responseCo...
5     {"id_":1,"size_":17,"maxDepth_":12,"responseCo...
6     {"id_":1,"size_":31,"maxDepth_":12,"responseCo...
7     {"id_":1,"size_":31,"maxDepth_":12,"responseCo...
8     {"id_":1,"size_":26,"maxDepth_":12,"responseCo...
9     {"id_":1,"size_":26,"maxDepth_":12,"responseCo...
10    {"id_":1,"size_":25,"maxDepth_":12,"responseCo...
11    {"id_":1,"size_":17,"maxDepth_":12,"responseCo...
12    {"id_":1,"size_":27,"maxDepth_":12,"responseCo...
13    {"id_":1,"size_":18,"maxDepth_":12,"responseCo...
14    {"id_":1,"size_":25,"maxDepth_":12,"responseCo...
15    {"id_":1,"size_":25,"maxDepth_":12,"responseCo...
16    {"id_":1,"size_":28,"maxDepth_":12,"responseCo...
17    {"id_":1,"size_":29,"maxDepth_":12,"respon

In [41]:
type(str("False").lower() in ['true', '1'])

bool

In [47]:
predictions = TDDecisionForestPredict(
        object=model,
        newdata=scaled_train.result,
        accumulate=target_name,
        id_column=entity_key,
        output_prob=True,
        output_responses=['0', '1']
    )

In [49]:
predictions.result.dtypes

COLUMN NAME,TYPE
PatientId,int
prediction,int
prob_0,float
prob_1,float
HasDiabetes,int


In [50]:
ClassificationEvaluator_obj = ClassificationEvaluator(
        data=predictions.result,
        observation_column=target_name,
        prediction_column='prediction',
        num_labels=2
    )

In [51]:
metrics_pd = ClassificationEvaluator_obj.output_data.to_pandas()

evaluation = {
        'Accuracy': '{:.2f}'.format(metrics_pd.MetricValue[0]),
        'Micro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[1]),
        'Micro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[2]),
        'Micro-F1': '{:.2f}'.format(metrics_pd.MetricValue[3]),
        'Macro-Precision': '{:.2f}'.format(metrics_pd.MetricValue[4]),
        'Macro-Recall': '{:.2f}'.format(metrics_pd.MetricValue[5]),
        'Macro-F1': '{:.2f}'.format(metrics_pd.MetricValue[6]),
        'Weighted-Precision': '{:.2f}'.format(metrics_pd.MetricValue[7]),
        'Weighted-Recall': '{:.2f}'.format(metrics_pd.MetricValue[8]),
        'Weighted-F1': '{:.2f}'.format(metrics_pd.MetricValue[9]),
}
evaluation

{'Accuracy': '0.77',
 'Micro-Precision': '0.77',
 'Micro-Recall': '0.77',
 'Micro-F1': '0.77',
 'Macro-Precision': '0.76',
 'Macro-Recall': '0.72',
 'Macro-F1': '0.73',
 'Weighted-Precision': '0.76',
 'Weighted-Recall': '0.77',
 'Weighted-F1': '0.76'}

In [53]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cf, img_filename):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(cf, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(cf.shape[0]):
        for j in range(cf.shape[1]):
            ax.text(x=j, y=i, s=cf[i, j], va='center',
                    ha='center', size='xx-large')
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def plot_roc_curve(roc_out, img_filename):
    import matplotlib.pyplot as plt
    auc = roc_out.result.to_pandas().reset_index()['AUC'][0]
    roc_results = roc_out.output_data.to_pandas()
    plt.plot(roc_results['fpr'], roc_results['tpr'],
             color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % 0.27)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()

In [55]:
cm = confusion_matrix(predictions.result.to_pandas()[
                          'HasDiabetes'], predictions.result.to_pandas()['prediction'])

plot_confusion_matrix(
        cm, "confusion_matrix")

<Figure size 750x750 with 0 Axes>

In [57]:
roc_out = ROC(
        data=predictions.result,
        probability_column='prob_1',
        observation_column=target_name,
        positive_class='1',
        num_thresholds=1000
    )

plot_roc_curve(roc_out, "roc_curve")

<Figure size 640x480 with 0 Axes>

In [58]:
try:
        model_pdf = model.result.to_pandas()['classification_tree']
        feature_importance = compute_feature_importance(model_pdf)
        feature_importance_df = pd.DataFrame(
            list(feature_importance.items()), columns=['Feature', 'Importance'])
        plot_feature_importance(
            feature_importance, "feature_importance_eval")
except:
        feature_importance = {}

<Figure size 640x480 with 0 Axes>

In [60]:
predictions_pdf = predictions.result.to_pandas(all_rows=True).rename(
        columns={"prediction": target_name}).astype(int)
predictions_pdf

,PatientId,HasDiabetes,prob_0,prob_1,HasDiabetes
0,221,1,0,0,1
1,265,0,0,0,0
2,461,0,0,0,0
3,713,0,0,0,0
4,242,0,0,0,1
...,...,...,...,...,...
763,46,0,0,0,0
764,227,1,0,0,1
765,717,0,0,0,0
766,225,0,0,0,0


In [65]:
predictions = TDDecisionForestPredict(
        object=model,
        newdata=scaled_train.result,
        id_column=entity_key,
        output_prob=True,
        output_responses=['0', '1']
    )
predictions_pdf = predictions.result.to_pandas(all_rows=True).rename(
        columns={"prediction": target_name}).astype(int)
predictions_pdf = pd.DataFrame(predictions_pdf, columns=[target_name])

In [66]:
predictions_pdf

,HasDiabetes
0,1
1,0
2,0
3,0
4,0
...,...
763,0
764,1
765,0
766,0


In [31]:
display_analytic_functions()


List of available functions:

	Analytics Database Functions:
		* MODEL SCORING functions:
			 1. DecisionTreePredict
			 2. GLMPredictPerSegment
			 3. KMeansPredict
			 4. NaiveBayesPredict
			 5. OneClassSVMPredict
			 6. SVMPredict
			 7. TDDecisionForestPredict
			 8. TDGLMPredict
			 9. TDNaiveBayesPredict
			 10. XGBoostPredict
		* FEATURE ENGINEERING TRANSFORM functions:
			 1. Antiselect
			 2. BincodeFit
			 3. BincodeTransform
			 4. ColumnTransformer
			 5. Fit
			 6. NonLinearCombineFit
			 7. NonLinearCombineTransform
			 8. OneHotEncodingFit
			 9. OneHotEncodingTransform
			 10. OrdinalEncodingFit
			 11. OrdinalEncodingTransform
			 12. Pivoting
			 13. PolynomialFeaturesFit
			 14. PolynomialFeaturesTransform
			 15. RandomProjectionFit
			 16. RandomProjectionMinComponents
			 17. RandomProjectionTransform
			 18. RowNormalizeFit
			 19. RowNormalizeTransform
			 20. ScaleFit
			 21. ScaleTransform
			 22. TargetEncodingFit
			 23. TargetEncodingTransform
			 24. TFI

In [70]:
DataFrame.from_query(f"""
        SELECT 
            * 
        FROM  pima_patient_predictions
        WHERE job_id = '9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb'
""")

job_id,PatientId,HasDiabetes,json_report
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,2,1,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,4,0,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,5,0,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,6,1,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,8,0,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,9,1,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,7,1,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,3,1,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,1,0,
9eaf64a6-d044-4c7b-8ad1-ad06f6a63efb,0,1,


In [73]:
DataFrame.from_query(f"""
        SELECT 
            * 
        FROM  "model_07f434ff-2a0a-45c6-8a65-d65b18cfc4aa"
""")

task_index,tree_num,tree_order,classification_tree
21,0,0,"{""id_"":1,""size_"":15,""maxDepth_"":12,""responseCounts_"":{""0"":11,""1"":4},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.448980,""attr_"":""Age"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.391111,""scoreImprove_"":0.391111,""leftNodeSize_"":11,""rightNodeSize_"":4},""leftChild_"":{""id_"":2,""size_"":11,""maxDepth_"":11,""label_"":""0"",""responseCounts_"":{""0"":11},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild_"":{""id_"":3,""size_"":4,""maxDepth_"":11,""label_"":""1"",""responseCounts_"":{""1"":4},""nodeType_"":""CLASSIFICATION_LEAF""}}"
22,0,0,"{""id_"":1,""size_"":4,""maxDepth_"":12,""responseCounts_"":{""0"":3,""1"":1},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.153846,""attr_"":""NumTimesPrg"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.375000,""scoreImprove_"":0.375000,""leftNodeSize_"":3,""rightNodeSize_"":1},""leftChild_"":{""id_"":2,""size_"":3,""maxDepth_"":11,""label_"":""0"",""responseCounts_"":{""0"":3},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild_"":{""id_"":3,""size_"":1,""maxDepth_"":11,""label_"":""1"",""responseCounts_"":{""1"":1},""nodeType_"":""CLASSIFICATION_LEAF""}}"
26,0,0,"{""id_"":1,""size_"":8,""maxDepth_"":12,""responseCounts_"":{""0"":2,""1"":6},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.824074,""attr_"":""SkinThick"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.375000,""scoreImprove_"":0.208333,""leftNodeSize_"":3,""rightNodeSize_"":5},""leftChild_"":{""id_"":2,""size_"":3,""maxDepth_"":11,""responseCounts_"":{""1"":1,""0"":2},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.266972,""attr_"":""DiPedFunc"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.444444,""scoreImprove_"":0.166667,""leftNodeSize_"":1,""rightNodeSize_"":2},""leftChild_"":{""id_"":4,""size_"":1,""maxDepth_"":10,""label_"":""1"",""responseCounts_"":{""1"":1},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild_"":{""id_"":5,""size_"":2,""maxDepth_"":10,""label_"":""0"",""responseCounts_"":{""0"":2},""nodeType_"":""CLASSIFICATION_LEAF""}},""rightChild_"":{""id_"":3,""size_"":5,""maxDepth_"":11,""label_"":""1"",""responseCounts_"":{""1"":5},""nodeType_"":""CLASSIFICATION_LEAF""}}"
13,0,0,"{""id_"":1,""size_"":9,""maxDepth_"":12,""responseCounts_"":{""0"":2,""1"":7},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.846154,""attr_"":""NumTimesPrg"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.345679,""scoreImprove_"":0.345679,""leftNodeSize_"":7,""rightNodeSize_"":2},""leftChild_"":{""id_"":2,""size_"":7,""maxDepth_"":11,""label_"":""1"",""responseCounts_"":{""1"":7},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild_"":{""id_"":3,""size_"":2,""maxDepth_"":11,""label_"":""0"",""responseCounts_"":{""0"":2},""nodeType_"":""CLASSIFICATION_LEAF""}}"
0,0,0,"{""id_"":1,""size_"":13,""maxDepth_"":12,""responseCounts_"":{""0"":7,""1"":6},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.605634,""attr_"":""PlGlcConc"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.497041,""scoreImprove_"":0.497041,""leftNodeSize_"":7,""rightNodeSize_"":6},""leftChild_"":{""id_"":2,""size_"":7,""maxDepth_"":11,""label_"":""0"",""responseCounts_"":{""0"":7},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild_"":{""id_"":3,""size_"":6,""maxDepth_"":11,""label_"":""1"",""responseCounts_"":{""1"":6},""nodeType_"":""CLASSIFICATION_LEAF""}}"
2,0,0,"{""id_"":1,""size_"":4,""maxDepth_"":12,""responseCounts_"":{""0"":1,""1"":3},""nodeType_"":""CLASSIFICATION_NODE"",""split_"":{""splitValue_"":0.292254,""attr_"":""PlGlcConc"",""type_"":""CLASSIFICATION_NUMERIC_SPLIT"",""score_"":0.375000,""scoreImprove_"":0.375000,""leftNodeSize_"":1,""rightNodeSize_"":3},""leftChild_"":{""id_"":2,""size_"":1,""maxDepth_"":11,""label_"":""0"",""responseCounts_"":{""0"":1},""nodeType_"":""CLASSIFICATION_LEAF""},""rightChild